In [2]:
import numpy as np


# **Definition**

Una transformación de Givens es una rotación en un plano definida por un ángulo $\theta$ que actúa sobre 2 coordenadas de un vector y 2 filas/columnas de una matriz.


Para definir la rotación de Givens en su forma matricial, se necesitan los indices $i,j$ tal que $i \neq j$, la forma matricial es una matriz identidad modificada:

$$
G(i,j,\theta) =

\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & c & \dots & -s & 0 \\
0 & \vdots & 1 & \vdots & 0 \\
0 & s & \dots & c & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & cos(\theta) & \dots & -sin(\theta) & 0 \\
0 & \vdots & 1 & \vdots & 0 \\
0 & sin(\theta) & \dots & cos(\theta) & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}

$$

---

# **Use Case 01: vector**
Dado un vector $\vec{v} \in 2 \times 1$, queremos aplicar una rotación tal que el segundo elemento se vuelva cero, es decir:

$$
\begin{bmatrix}
c & -s \\
s & c
\end{bmatrix}
\begin{bmatrix}
a \\
b 
\end{bmatrix}
=
\begin{bmatrix}
r \\
0 
\end{bmatrix}
$$

si desarrollamos la multiplicación:
$$
\begin{bmatrix}
(ca) + (-sb) \\
(sa) + (cb)
\end{bmatrix}
=
\begin{bmatrix}
r \\
0 
\end{bmatrix}
$$

Ahora si conveniente definimos $c = \frac{a}{r}$ y $s = - \frac{b}{r}$:

$$
\begin{bmatrix}
(\frac{a}{r}a) + (\frac{b}{r}b) \\
(-\frac{b}{r}a) + (\frac{a}{r}b)
\end{bmatrix}
=
\begin{bmatrix}
r \\
0 
\end{bmatrix}
$$

Y notamos que el segundo elemento de la matriz se cancela lo cual cumple nuestro objetivo inicial y el primer elemento queda de la siguiente forma:

$$
\begin{bmatrix}
\frac{(a^2 + b^2)}{r} \\
0
\end{bmatrix}
=
\begin{bmatrix}
r \\
0 
\end{bmatrix}
$$

Donde:

$$
\begin{align*}
\frac{(a^2 + b^2)}{r} &= r \\
a^2 + b^2 &= r^2 \\
r &= \sqrt{a^2 + b^2}
\end{align*}
$$

Es decir $r$ es la longitud del vector inicial, lo cual tiene sentido porque la transformación de Givens es una rotación y si deseamos dejar solo 1 elemento del vector con una rotación, el único elemento que queda tiene que tener toda la magnitud del vector inicial.

In [3]:
a = 3
b = 4
vector = np.array([a, b])
r = np.linalg.norm(vector)
c = a / r
s = -b / r
G = np.array([[c, -s], [s, c]])

print("Original vector:\n", vector)
print("Norm of original vector (r):", r)
print("Givens rotation matrix G:\n", G)
rotated_vector = G @ vector
print("Rotated vector (should be [r, 0]):\n", rotated_vector)



Original vector:
 [3 4]
Norm of original vector (r): 5.0
Givens rotation matrix G:
 [[ 0.6  0.8]
 [-0.8  0.6]]
Rotated vector (should be [r, 0]):
 [ 5.00000000e+00 -2.22044605e-16]


---

# **Use Case 02: Hesenberg Matrix**

Ahora nuestro objetivo es transformar una matriz de Hesenberg en una matriz triagunlar. es decir partimos de $H$:

$$
H =
\begin{bmatrix}
h_{11} & h_{12} & h_{13} & h_{14} \\
h_{21} & h_{22} & h_{23} & h_{24} \\
0      & h_{32} & h_{33} & h_{34} \\
0      & 0      & h_{43} & h_{44}
\end{bmatrix}
$$

y queremos aplicar una serie de transformaciones de Givens para eliminar los factores $h_{21},h_{32}, h_{43}$.

Ahora para eliminar el elemento $h_{21}$ se construye una rotación de Givens tal que:
$$
G(1,2,\theta)
\begin{bmatrix}
h_{11} \\
h_{21}
\end{bmatrix}
=
\begin{bmatrix}
r \\
0
\end{bmatrix}
$$

Luego se aplica la rotación a toda la matriz original.

## Algoritmo:

1. Ir recorriendo las subdiagonales.
2. para par de posiciones $(i+1,i)$, construimos la rotación con los valores $h_{ii}, h_{i+1,i}$
3. Aplicar $G(i,i+1,\theta)$

In [4]:
from scipy.linalg import hessenberg

def create_random_hessenberg_matrix(n: int) -> np.ndarray:
    if n < 3:
        raise ValueError("La dimensión debe ser un entero positivo.")
    
    A = np.random.rand(n, n)
    H = hessenberg(A)
    return H

n = 4
hessenberg_matrix = create_random_hessenberg_matrix(n)
print("Matriz de Hessenberg de 5x5:")
print(hessenberg_matrix)

Matriz de Hessenberg de 5x5:
[[ 0.69473215 -1.25475191 -0.80243156  0.27490136]
 [-0.82869893  0.98807626  1.07651805 -0.24058264]
 [ 0.          0.69450867  0.54240152 -0.18665959]
 [ 0.          0.         -0.64347058 -0.31081498]]


In [ ]:
def givens_rotation(a,b):
    """
    Calcula los valores de seno y coseno para una rotación de Givens.
    """
    if b == 0:
        c = 1.0
        s = 0.0
    else:
        hyp = np.hypot(a, b)
        c = a / hyp
        s = -b / hyp
    return c, s

def hessenberg_to_upper_triangular(H):
    """
    Transforma una matriz de Hessenberg en una matriz triangular superior
    usando rotaciones de Givens.
    """
    n = H.shape[0]
    R = H.copy().astype(np.float64)  
    
    for row in range(n - 1):
        diagonal = R[row, row]
        subdiagonal = R[row + 1, row]
        
        c, s = givens_rotation(diagonal, subdiagonal)
        
        for column in range(row, n):
            temp_diag = R[row, column]
            temp_subdiag = R[row + 1, column]
            R[row, column] = c * temp_diag - s * temp_subdiag
            R[row + 1, column] = s * temp_diag + c * temp_subdiag
            
    return R

H_example = create_random_hessenberg_matrix(4)

print("Matriz de Hessenberg original:")
print(np.round(H_example,2))
print("\n" + "-"*30 + "\n")

# Transformar a triangular superior
R_triangular = hessenberg_to_upper_triangular(H_example)

print("Matriz triangular superior resultante:")
print(np.round(R_triangular,2))

Matriz de Hessenberg original:
[[ 0.83 -0.9   0.35 -0.31]
 [-0.24  1.1  -0.96 -0.08]
 [ 0.   -0.71  0.27  0.28]
 [ 0.    0.    0.5   0.5 ]]

------------------------------

Matriz triangular superior resultante:
[[ 0.86 -1.17  0.6  -0.27]
 [ 0.    1.08 -0.8  -0.31]
 [ 0.    0.    0.6   0.35]
 [ 0.    0.    0.   -0.36]]


# **Por que no hay construcción explicita de G**

1. notar que el bucle que recorre la diagonal `for i in range(n - 1)` tiene rango $n-1$ porque es necesario eliminar elementos hasta la penúltima fila.


2. El código no construye la matriz de Givens porque esta por definición es una modificación de la Identidad, es decir la mayor parte de la multiplicación no afecta la matriz H, solo se afectan las filas `i` y `i+1`:

$$
G_i H =
\begin{bmatrix}
\vdots \\
\text{row }i \\
\text{row }i+1 \\
\vdots
\end{bmatrix}
=
\begin{bmatrix}
\vdots \\
c \cdot (\text{row }i) - s \cdot (\text{row }i+1) \\
s \cdot (\text{row }i) + c \cdot (\text{row }i+1) \\
\vdots
\end{bmatrix}
$$

Esto es precisamente lo que se hace en el segundo bucle.
